# Model training 🏋
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nasaharvest/openmapflow/blob/main/openmapflow/notebooks/train.ipynb)

Example notebook for training models.

# 1. Load project

If you don't already have one, obtain a Github Personal Access Token using the steps [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token). Save this token somewhere private.

In [ ]:
!pip install "ipywidgets>=7,<8" -q # https://github.com/googlecolab/colabtools/issues/3020

In [ ]:
from ipywidgets import Password, Text, VBox
inputs = [
      Password(description="Github Token:"),
      Text(description='Github Email:'),
      Text(description='Github User:'),
      Text(description='Github URL:'),
]
VBox(inputs)

In [ ]:
token = inputs[0].value
email = inputs[1].value
username = inputs[2].value
github_url = inputs[3].value

!git config --global user.email $username
!git config --global user.name $email
!git clone {github_url.replace("https://", f"https://{username}:{token}@")}

In [ ]:
from pathlib import Path
import os
openmapflow_yaml_path = input("Path to openmapflow.yaml: ")
%cd {Path(openmapflow_yaml_path).parent}

!pip install -r requirements.txt -q
!pip install pyyaml==5.4.1 -q

# 2. Pull in latest data

In [ ]:
!dvc pull -q

In [ ]:
!openmapflow datasets

# 3. Train model

In [ ]:
os.environ["MODEL_NAME"] = input("MODEL_NAME=")

In [ ]:
!python train.py --model_name $MODEL_NAME --epoch 2

# 4. Push model to dvc and git

In [ ]:
# Record test metrics
!python evaluate.py --model_name $MODEL_NAME

In [ ]:
!dvc commit -q
!dvc push -q

In [ ]:
# Push changes to github
!git checkout -b"$MODEL_NAME"
!git add .
!git commit -m "$MODEL_NAME"
!git push --set-upstream origin "$MODEL_NAME"

Create a Pull Request so the model can be merged into the main branch.